In [2]:
import numpy as np
from typing import List
from abc import ABC, abstractmethod

## 0. Setup

I used TFX package to make EDA and feature transformation. It is a powerful tool to automate ML pipelines and build a preprocessing layer which is saved as a submodel. 

You can setup conda environment like this: 

```
conda create -n tfx_pipeline python=3.8
conda activate tfx_pipeline
pip --default-timeout=5000 --use-deprecated=legacy-resolver install tfx  scikit-learn
```

Or you can use the freezed pip list from *requirements.txt* with python 3.8 or 3.9.

## 1. Counting Islands

In [8]:
class IslandsCounter:
    def __call__(self, grid: List[List[int]]) -> int:
        visited = set()
        res = 0
        N = len(grid)
        M = len(grid[0])
        for n in range(N):
            for m in range(M):
                if grid[n][m] == 0 or (n, m) in visited:
                    continue
                else:
                    res += 1
                    que = []
                    visited.add((n, m))
                    que.append((n, m))
                    while que:
                        a, b = que.pop()
                        for i, j in [(a - 1, b), (a + 1, b ), (a, b + 1), (a, b - 1)]:
                            if -1 < i < N and -1 < j < M and \
                                grid[i][j] == 1 and (i, j) not in visited:
                                visited.add((i, j))
                                que.append((i, j))
        return res

In [16]:
tests = [[[0, 1, 0], 
          [0, 0, 0], 
          [0, 1, 1]], 
         [[0, 0, 0, 1], 
          [0, 0, 1, 0],
          [0, 1, 0, 0]],
         [[0, 0, 0, 1],
          [0, 0, 1, 1],
          [0, 1, 0, 1]]
        ]

counter = IslandsCounter()

for grid in tests: 
    print(counter(grid))

2
3
2


## 2. Regression on the tabular data

#### 1. Run train script. The default parameter for --train_data is './data/train.csv'.  Results will be saved in the results.csv file.

In [4]:
! python run_train.py

2024-07-19 13:47:46.116956: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:47:46.175369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 13:47:46.175417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 13:47:46.176852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 13:47:46.185130: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:47:47.328027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

#### 2. After run_train script is finished you may use new_model and new_tfx preprocessing layer to get fresh predictions.

In [6]:
! python run_predict.py --data_path='./data_for_tfx/val.csv' --model_path='./new_model' --tfx_root='./new_tfx'

2024-07-19 13:54:53.083816: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:54:53.128243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 13:54:53.128277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 13:54:53.129792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 13:54:53.137028: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:54:54.207615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

#### 3. Or you can try cached_model and cached_tfx preprocessing layer which gave me the smallest mean_abs_percentage error.

In [7]:
! python run_predict.py --data_path='./data_for_tfx/val.csv' --model_path='./cached_model' --tfx_root='./cached_tfx'

2024-07-19 13:55:28.459589: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:55:28.502828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 13:55:28.502875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 13:55:28.504334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 13:55:28.511527: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:55:29.563093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

#### 4. Results of hidden_test.csv is stored in the hidden_test_results.csv. I renamed results.csv for these inputs by hands. 

In [3]:
! python run_predict.py --data_path='./data/hidden_test.csv' --model_path='./cached_model' --tfx_root='./cached_tfx'

2024-07-19 13:46:40.230935: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:46:40.286414: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 13:46:40.286452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 13:46:40.288074: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 13:46:40.296638: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:46:41.363879: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

#### 5. Further researches

I have finished the base model prototype. What could be done next? 

- Find optimal hyperparameters for the training process: learning rate, decay rate, epoch count, and gradient descent algorithms.
- Neural architecture search: Make the net deeper and change batch__norm_block inner and output width. 

Both steps could be done via Katib (standalone or as a part of Kubeflow / Vertex). It is possible to configs the training job and experiment for this microservice. The main advantage of Katib is the ability to run several training jobs simultaneously and automate hyperparameters search via Bayesian optimization. 

- After we find the most optimal learning and net hyperparameters we can add Train, ModelEstimation, and Pusher components to the transform pipeline and create continuous building and training pipelines if it is possible to get a continuous amount of labeled data for the training process. We can run such pipelines locally, but the preferable way is to use Kubeflow pipelines, cause we can monitor data, and model shifts via artefacts visualization easily. 

- If we want an inference service for this model we can use KServe. It can help leverage GPU batch computation by creating a request queue sidecar service which will generate batches from atomic requests of users. 

## 3. MNIST classifier. OOP

In [18]:
# Define the interface
class DigitClassificationInterface(ABC):
    
    def __init__(self, shape:tuple):
        self.shape = shape
    
    @abstractmethod
    def predict(self, image: np.ndarray) -> int:
        pass
    
    def preprocess(self, image: np.ndarray) -> np.ndarray:
        if type(image) != np.ndarray:
            raise TypeError("image must be a numpy array")
            
        prep_image = self._preprocess(image)
        
        if prep_image.shape != self.shape:
            raise ValueError(f"image shape must match {self.shape}, but got {prep_image.shape}")
            
        return prep_image
            
    
    @abstractmethod
    def _preprocess(self, image: np.ndarray) -> int:
        pass
    

# CNN Model implementation
class CNNModel(DigitClassificationInterface):
    
    def __init__(self):
        super().__init__((28, 28, 1))
    
    def _preprocess(self, image: np.ndarray) -> np.ndarray:
        return image
    
    def predict(self, image: np.ndarray) -> int:
        # For demonstration, let's return a random number as placeholder
        return np.random.randint(0, 10)

# Random Forest Model
class RFModel(DigitClassificationInterface):
    def __init__(self):
        # Initialize the random forest model
        super().__init__((784,))
    
    def _preprocess(self, image:np.ndarray) -> np.ndarray:
        return image.flatten()

    def predict(self, image: np.ndarray) -> int:
        return np.random.randint(0, 10)

# Random Model
class RandomModel(DigitClassificationInterface):
    
    def __init__(self):
        super().__init__((10, 10, 1))
    
    def _preprocess(self, image:np.ndarray) -> np.ndarray:
        return image[9:19, 9:19]
    
    def predict(self, image: np.ndarray) -> int:
        # Here we can ignore the input and return a random value
        return np.random.randint(0, 10)

# Digit Classifier
class DigitClassifier:
    def __init__(self, algorithm: str):
        if algorithm == 'cnn':
            self.model = CNNModel()
        elif algorithm == 'rf':
            self.model = RFModel()
        elif algorithm == 'rand':
            self.model = RandomModel()
        else:
            raise ValueError(f"Unsupported algorithm: {algorithm}")
            
    def train(self, x:np.ndarray, labels:np.ndarray):
        raise NotImplementedError()

    def predict(self, image: np.ndarray) -> int:
        prepared_image = self.model.preprocess(image)
        return self.model.predict(prepared_image)

In [19]:
# Create a random 28x28 image
image = np.random.rand(28, 28, 1)

# Create a DigitClassifier with a specific algorithm
classifier = DigitClassifier(algorithm='cnn')
prediction = classifier.predict(image)
print(f"CNN Prediction: {prediction}")

classifier = DigitClassifier(algorithm='rf')
prediction = classifier.predict(image)
print(f"Random Forest Prediction: {prediction}")

classifier = DigitClassifier(algorithm='rand')
prediction = classifier.predict(image)
print(f"Random Model Prediction: {prediction}")

CNN Prediction: 9
Random Forest Prediction: 8
Random Model Prediction: 1
